In [3]:
from extract_product_info import product_info
from update_reference_json import update_reference_json_from_catalog
from update_component_name import update_component
from update_property_blocks import update_property

In [2]:
catalog_json = product_info()

Paste product specification info (end with a blank line):

--- URL content provided ---

DOGCOM 1550mAh 4S 14.8V 150C UCELL series LiPo Battery - XT60
Price:Current price$39.99
Brand: DOGCOM UCELL series Capacity: 1550mAh  Configuration: 14.8V / 4S1P / 4cell Continuous Discharge Rate: 150C Max Discharge Rate: 300C Dimensions: T*W*L=28mm x 39mm x 80mm Weight: about 181g Power: 22.94Wh Charge Plug: JST-XHR Discharge Plug: XT60

------ Product Info Output ------

{'price': '$39.99', 'brand': 'DOGCOM UCELL series', 'capacity': '1550mAh', 'configuration': '14.8V / 4S1P / 4cell', 'continuous_discharge_rate': '150C', 'max_discharge_rate': '300C', 'dimensions': 'T*W*L=28mm x 39mm x 80mm', 'weight': 'about 181g', 'power': '22.94Wh', 'charge_plug': 'JST-XHR', 'discharge_plug': 'XT60'}


In [3]:
# ACM source file and the JSON I created from its Property blocks to be updated (both are used as reference to guide the LLM)
acm_file_name = "battery_test.acm" 

reference_json = {
    "LENGTH": "74.0",
    "MODEL": "Turnigy Graphene 1000mAh 2S 75C",
    "NUMBER_OF_CELLS": "2S1P",
    "THICKNESS": "17",
    "DISCHARGE_PLUG": "XT60",
    "VOLTAGE": "7.4",
    "CAPACITY": "1000",
    "WIDTH": "35",
    "BATTERY_TYPE": "UAV",
    "COST": "9.54",
    "WEIGHT": "0.084",
    "CONT_DISCHARGE_RATE": "75",
    "PEAK_DISCHARGE_RATE": "150",
    "CHEMISTRY_TYPE": "LiPo",
    "DESCRIPTION": "",
    "MODELED_BY": "",
    "PACK_RESISTANCE": "11.0",
    "ROTATION": {
        "Default": 0,
        "Maximum": 360,
        "Minimum": 0,
        "AssignedValue": 0,
    }
}


In [4]:
final_json = update_reference_json_from_catalog(catalog_json, reference_json) 


------ Final Updated JSON ------

{'LENGTH': '80', 'MODEL': 'DOGCOM UCELL series 1550mAh 4S1P / 4cell', 'NUMBER_OF_CELLS': '4S1P', 'THICKNESS': '28mm', 'DISCHARGE_PLUG': 'XT60', 'VOLTAGE': '14.8', 'CAPACITY': '1550', 'WIDTH': '39mm', 'BATTERY_TYPE': 'UAV', 'COST': '39.99', 'WEIGHT': '181g', 'CONT_DISCHARGE_RATE': '150C', 'PEAK_DISCHARGE_RATE': '300C', 'CHEMISTRY_TYPE': 'LiPo', 'DESCRIPTION': '', 'MODELED_BY': '', 'PACK_RESISTANCE': '11', 'ROTATION': {'Default': 0, 'Maximum': 360, 'Minimum': 0, 'AssignedValue': 0}}


In [5]:
updated_component_tag = update_component(acm_file_name, final_json)


------ Updated component Name ACM ------

<Component xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" SchemaVersion="2.5" Name="DOGCOM_UCELL_series_1550mAh_4S1P_4cell" Version="" xmlns="avm">


In [6]:
update_properties = update_property(acm_file_name, final_json)

--- Updated property ACM XML ---
<Property xmlns:q9="avm" xsi:type="q9:PrimitiveProperty" Name="LENGTH" OnDataSheet="false" ID="property.id-e9ddffc4-a90f-11eb-bbb5-784f4367106c" xmlns="">
    <Value ID="id-e9ddffc4-a90f-11eb-bbb5-784f4367106c" DimensionType="Scalar" DataType="String">
      <ValueExpression xsi:type="q9:FixedValue">
        <Value>80</Value>
      </ValueExpression>
    </Value>
  </Property>
<Property xmlns:q10="avm" xsi:type="q10:PrimitiveProperty" Name="MODEL" OnDataSheet="false" ID="property.id-e9de263e-a90f-11eb-bbb5-784f4367106c" xmlns="">
    <Value ID="id-e9de263e-a90f-11eb-bbb5-784f4367106c" DimensionType="Scalar" DataType="String">
      <ValueExpression xsi:type="q10:FixedValue">
        <Value>DOGCOM UCELL series 1550mAh 4S1P / 4cell</Value>
      </ValueExpression>
    </Value>
</Property>
<Property xmlns:q11="avm" xsi:type="q11:PrimitiveProperty" Name="NUMBER_OF_CELLS" OnDataSheet="false" ID="property.id-e9ddf2f4-a90f-11eb-bbb5-784f4367106c" xmlns="">
   

In [9]:
import re
import json

# load the reference ACM XML content from the file (default reading mode)
with open(acm_file_name) as file:
    xml_content = file.read()

# Extract the opening tag
component_tag_match = re.search(r'<Component\b[^>]*>', xml_content)
component_tag = component_tag_match.group()

# Extract all property block
property_blocks = re.findall(r'(<Propert[^>]*>.*?</Property>)', xml_content, flags=re.DOTALL)
    
# replace original Component opening tag with updated one
xml_content_updated = xml_content.replace(component_tag, updated_component_tag)

# replace all original Property blocks with updated ones, one by one
for original_prop, updated_prop in zip(property_blocks, update_properties):
    xml_content_updated = xml_content_updated.replace(original_prop, updated_prop)

# write the fully updated ACM content to a file 
base_name = json.dumps(final_json.get("MODEL")).strip('"')
safe_name = re.sub(r'[\\/:"*?<>|]+', '_', base_name) # replace with underscore for characters invalid for filenames (slash, colons, etc.)
output_path = f"{safe_name}.acm"
with open(output_path, "w") as file:
    file.write(xml_content_updated)

print(f"Fully updated ACM written to: {output_path}")

Fully updated ACM written to: DOGCOM UCELL series 1550mAh 4S1P _ 4cell.acm
